In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import f
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


This problem set explores the value-growth effect on two dimensions: 

1) how these returns vary over the business cycle and 

2) whether characteristics or covariances better capture their returns. 

We will also look at the characteristics vs. covariance debate for another anomaly – momentum. Reading the Lakonishok, Shleifer, and Vishny (1994) and Daniel and Titman (1997) articles will be most useful.

In order to proceed you need Microsoft Excel and file “Problem_Set5.xls”. This file contains three spreadsheets:

1)	The factor portfolios calculated by Fama and French, a dummy variable indicating recessionary periods as defined by the National Bureau of Economic Research.  In addition, the file contains monthly T-bill returns. The factor portfolios include the RMRF portfolio (market minus T-bill rate), SMB portfolio (size factor), HML portfolio (book-to-market factor), and UMD portfolio (momentum factor).

2)	The 25 size and BE/ME sorted portfolios of Fama and French. The value-weighted average returns, average size, and average BE/ME characteristic of each of these portfolios over time are provided.

3)	The 25 size and Momentum (past 2-to-12 month return) sorted portfolios of Fama and French. The value-weighted average returns, average size, and average past return characteristic of each of these portfolios over time are provided.




# Data Processing

In [ ]:
#--------------------Read subsheet 1--------------------
sheet1=pd.read_excel("Problem_Set5.xls", sheet_name='Fama-French factors', 
                     skiprows=2, index_col=None, na_values=-999)
sheet1["Date"] = pd.to_datetime(sheet1["Unnamed: 0"], format="%Y%m")
factors = sheet1.set_index("Date")
factors = factors.loc[factors.index.dropna()]
factors = factors.drop("Unnamed: 0", axis = 1)

#--------------------Read subsheet 2--------------------
sheet2_return=pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_BEME_Portfolios', 
                     skiprows=2, index_col=None, usecols="A:Z", na_values=-999)
sheet2_return["Date"] = pd.to_datetime(sheet2_return["BE/ME"], format="%Y%m")
s2_return = sheet2_return.set_index("Date")
s2_return = s2_return.loc[s2_return.index.dropna()]
s2_return = s2_return.drop("BE/ME", axis = 1)

sheet2_size = pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_BEME_Portfolios', 
                     skiprows=2, index_col=0, usecols="AB:BA", na_values=-999)
sheet2_size = sheet2_size.loc[sheet2_size.index.dropna()]
sheet2_size['Date'] = sheet2_size.index
sheet2_size['Date'] = pd.to_datetime(sheet2_size['Date'], format="%Y%m")
s2_size = sheet2_size.set_index("Date")

sheet2_BEME = pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_BEME_Portfolios', 
                     skiprows=2, index_col=0, usecols="BC:CB", na_values=-999)
sheet2_BEME = sheet2_BEME.loc[sheet2_BEME.index.dropna()]
sheet2_BEME['Date'] = sheet2_BEME.index
sheet2_BEME['Date'] = pd.to_datetime(sheet2_BEME['Date'], format="%Y")
s2_BEME = sheet2_BEME.set_index("Date")

#--------------------Read subsheet 3--------------------
sheet3_return=pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_212_Portfolios', 
                     skiprows=2, index_col=None, usecols="A:Z", na_values=-999)
sheet3_return["Date"] = pd.to_datetime(sheet3_return["Ret212"], format="%Y%m")
s3_return = sheet3_return.set_index("Date")
s3_return = s3_return.loc[s3_return.index.dropna()]
s3_return = s3_return.drop("Ret212", axis = 1)

sheet3_size = pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_212_Portfolios', 
                     skiprows=2, index_col=0, usecols="AB:BA", na_values=-999)
sheet3_size = sheet3_size.loc[sheet3_size.index.dropna()]
sheet3_size['Date'] = sheet3_size.index
sheet3_size['Date'] = pd.to_datetime(sheet3_size['Date'], format="%Y%m")
s3_size = sheet3_size.set_index("Date")

sheet3_ret212 = pd.read_excel("Problem_Set5.xls", sheet_name='25_Size_212_Portfolios', 
                     skiprows=2, index_col=0, usecols="BC:CB", na_values=-999)
sheet3_ret212 = sheet3_ret212.loc[sheet3_ret212.index.dropna()]
sheet3_ret212['Date'] = sheet3_ret212.index
sheet3_ret212['Date'] = pd.to_datetime(sheet3_ret212['Date'], format="%Y%m")
s3_ret212 = sheet3_ret212.set_index("Date")

# Merged datasets
# merge s2_return with sheet1(factors)
s2_return_s1 = pd.merge(s2_return, factors, how='inner', left_index=True, right_index=True)
s3_return_s1 = pd.merge(s3_return, factors, how='inner', left_index=True, right_index=True)

# Business cycle variation

## a)	Compute the average return on RMRF, SMB, HML, and UMD during recessions only. This can be done by running a regression of the returns of each factor portfolio on an intercept and a recession dummy variable. What does this tell you?  Do you believe SMB or HML is more related to risk or other sources based on these results?

For each of RMRF, SMB, HML, and UMD, we do the following regression
$$\text{Return} = \beta_0 + \beta_1 \mathbb1_{recession} + \epsilon,$$

In [ ]:
for i in factors.columns[0:5]:
  if i == 'RF':
    continue
  elif i == 'UMD':
    x = sm.add_constant(factors['Recession indicator (1=recession, 0=non-recesstion)'][6:])
    mod1 = sm.OLS(factors[i][6:], x, data = factors).fit()
    print(i, "beta0: ", round(mod1.params[0], 2),"beta1: ", round(mod1.params[1], 2), "average return:", \
          round(mod1.params[0] + mod1.params[1], 2), "p-value of beta0:", round(mod1.pvalues[0],2), "p-value of beta1:", round(mod1.pvalues[1],2))
  else:
    x = sm.add_constant(factors['Recession indicator (1=recession, 0=non-recesstion)'])
    mod1 = sm.OLS(factors[i], x, data = factors).fit()
    print(i, "beta0: ", round(mod1.params[0], 2),"beta1: ", round(mod1.params[1], 2), "average return:", \
          round(mod1.params[0] + mod1.params[1], 2), "p-value of beta0:", round(mod1.pvalues[0],2), "p-value of beta1:", round(mod1.pvalues[1],2))

Mkt-RF beta0:  0.9 beta1:  -1.33 average return: -0.43 p-value of beta0: 0.0 p-value of beta1: 0.0
SMB beta0:  0.26 beta1:  -0.26 average return: 0.01 p-value of beta0: 0.02 p-value of beta1: 0.31
HML beta0:  0.42 beta1:  -0.19 average return: 0.23 p-value of beta0: 0.0 p-value of beta1: 0.48
UMD beta0:  0.78 beta1:  -0.52 average return: 0.26 p-value of beta0: 0.0 p-value of beta1: 0.16


There are a few interesting observations from the above results.

For $\beta_0$ values:

*   All $\beta_0$ values are positive. Directionally this makes sense as the return is positive during non-recession period. Moreover, the intercept represents the risk of the asset itself, unrelated to other risk factors, i.e. return when all other factors are 0, and therefore higher risk should gives higher return; 
*   In terms of magnitude, we can see Mkt-RF has the highest positive value, followed by UMD, which indicates that Mkt-RF is most positively related to return, rollowed by the momentum factor. Interestingly, SMB and HML both have lower factor loads than the other two sources;
*   The p-values for $\beta_0$ shows how statistically significant this "self" risk factor is. The lower the p-values, the more statistically significant the values are. Therefore, we can see that all $\beta_0$ factors are quite statistically significant.

For $\beta_1$ values:

*   All $\beta_1$ values are negative. Intuitively this makes sense as return can be negative during recession;
*   Except for Mkt-RF, the p-values for SMB / HML/ UMD portoflios indicates that they are all statistically not significant and therefore unrelated to the risk. Between SMB and HML, SMB is more statistically significant, and it's more related to risk. In this sense, UMD and Mkt-RF both are more related to risk than SMB or HML.

For average return:

*  during the recessions, the market portfolio generates a negative return on average, while SMB / HML/ UMD portoflios hve positive returns on average.

## b)	Repeat part a) only for the smallest growth, smallest value, and largest growth and largest value portfolios (e.g., the “four corners” of the 25 Fama-French portfolios), and for the smallest losers, smallest winners, and largest losers and winners. Make sure you use the excess returns of these portfolios by subtracting the risk-free rate.  Do you notice any stronger cyclical relationships for these four portfolios?

In [ ]:
loop_list = [("Low", "smallest growth"),("High", "smallest value"),("Low.4","largest growth"), ("High.4","largest value")]

for i, j in loop_list:
  y = s2_return_s1[i] - s2_return_s1["RF"]
  x = sm.add_constant(s2_return_s1['Recession indicator (1=recession, 0=non-recesstion)'])
  mod1 = sm.OLS(y,x).fit()
  print(i, "beta0: ", round(mod1.params[0], 2),"beta1: ", round(mod1.params[1], 2), "average return:", \
          round(mod1.params[0] + mod1.params[1], 2), "p-value of beta0:", round(mod1.pvalues[0],2), "p-value of beta1:", round(mod1.pvalues[1],2))

Low beta0:  0.86 beta1:  -1.53 average return: -0.67 p-value of beta0: 0.04 p-value of beta1: 0.11
High beta0:  1.75 beta1:  -2.1 average return: -0.35 p-value of beta0: 0.0 p-value of beta1: 0.0
Low.4 beta0:  0.84 beta1:  -1.18 average return: -0.34 p-value of beta0: 0.0 p-value of beta1: 0.0
High.4 beta0:  1.17 beta1:  -1.29 average return: -0.12 p-value of beta0: 0.0 p-value of beta1: 0.05


In [ ]:
loop_list = [("Low", "smallest loser"),("High", "smallest winner"),("Low.4","largest loser"), ("High.4","largest winner")]

for i, j in loop_list:
  y = s3_return_s1[i] - s3_return_s1["RF"]
  x = sm.add_constant(s3_return_s1['Recession indicator (1=recession, 0=non-recesstion)'])
  mod1 = sm.OLS(y,x,missing = 'drop').fit()
  print(j, " beta0: ", round(mod1.params[0], 2), "beta1: ", round(mod1.params[1], 2), "average return:", \
        round(mod1.params[0] + mod1.params[1], 2), "p-value of beta1:", round(mod1.pvalues[1],2))

smallest loser  beta0:  0.85 beta1:  -1.2 average return: -0.34 p-value of beta1: 0.16
smallest winner  beta0:  1.99 beta1:  -1.97 average return: 0.02 p-value of beta1: 0.0
largest loser  beta0:  0.3 beta1:  -2.06 average return: -1.76 p-value of beta1: 0.01
largest winner  beta0:  1.25 beta1:  -1.61 average return: -0.36 p-value of beta1: 0.0


For the clyclical relationships, we look at parameters associated with $\beta_1$, since it's a regression on the recession variables. 

From the p-values of $\beta_1$ we can see that the smallest growth portfolio in the 25 Fama-French portoflios, and the smallest lowers portfolio both have high numbers, indicating that for these two portfolios the business cyclicality is not a statistically significant risk factor.

However, we can notice that all other portoflios have strong cyclical relationships. In fact, we can see that the relationships are much stronger than in part a) for portfolios SMB, HML, and UMD.

# Characteristics vs. Covariances

**25 Size and BE/ME portfolios.**

c)	Consider the following Fama-MacBeth cross-sectional regressions: 
$$
\begin{array}{ll}(1) & \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\mathrm{size}} \ln (\text { size })+\gamma_{\mathrm{B} / \mathrm{M}} \ln (\mathrm{BE} / \mathrm{ME})+\eta_{\mathrm{i} 1} \\ (2) & \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\mathrm{SMB}} \beta_{\mathrm{iSMB}}+\gamma_{\mathrm{HML}} \beta_{\mathrm{iHML}}+\eta_{\mathrm{i} 2} \\ (3) & \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\mathrm{size}} \ln (\text { size })+\gamma_{\mathrm{B} / \mathrm{M}} \ln (\mathrm{BE} / \mathrm{ME})+\gamma_{\mathrm{SMB}} \beta_{\mathrm{iSMB}}+\gamma_{\mathrm{HML}} \\ & \beta_{\mathrm{iHML}}+\eta_{\mathrm{i} 3}\end{array}
$$

where $\gamma_{0}, \gamma_{\mathrm{M}}, \gamma_{\mathrm{size}}, \gamma_{\mathrm{B} / \mathrm{M}}, \gamma_{\mathrm{SMB}}, \text { and }{\gamma}_{\mathrm{HML}}$ are regression parameters. $\beta_{\mathrm{iM}}, \beta_{\mathrm{iSMB}}, \text { and } \beta_{iHML}$ are betas with respect to the Fama-French factors SMB and HML and size and BE/ME are the average size and book to market ratio characteristics of the portfolio.

Estimate equations (1), (2), and (3) using the full sample of data and the Fama- MacBeth procedure. The following is a brief outline of the procedure:

1. Estimate $\beta_{\mathrm{iM}}, \beta_{\mathrm{iSMB}}, \text { and } \beta_{iHML}$ for each portfolio by running a time-series regression for each of the 25 portfolios on the Fama-French factors RMRF, SMB, and HML. Assume that the betas do not change over time; hence, you can estimate the betas using full-period OLS regressions.
2. Each month estimate the regressions using the month-by-month cross-section of realized returns on the 25 portfolios on their estimated betas and characteristics according to equations (1), (2), and (3).
(Hint: You have already estimated equation (1) in problem set 3! Hope you saved your program.)
3. Compute the time series average of the estimates of$\gamma_{0}, \gamma_{\mathrm{M}}, \gamma_{\mathrm{size}}, \gamma_{\mathrm{B} / \mathrm{M}}, \gamma_{\mathrm{SMB}}, \text { and }{\gamma}_{\mathrm{HML}}$. In addition, compute the standard error and t-stat of the time series averages in the style of Fama and MacBeth (1973).

In [ ]:
# part 1
beta_M = []
beta_SMB = []
beta_HML = []
for idx,i in enumerate(s2_return_s1.columns[0:25]):
  y = s2_return_s1[i] - s2_return_s1["RF"]
  x = sm.add_constant(s2_return_s1[["Mkt-RF", "SMB","HML"]])
  mod1 = sm.OLS(y,x,missing = 'drop').fit()
  beta_M.append(mod1.params[1])
  beta_SMB.append(mod1.params[2])
  beta_HML.append(mod1.params[3])
  print("portfolio #", idx+1, "beta_M: ",round(mod1.params[1],2), \
        "beta_SMB: ", round(mod1.params[2],2), "beta_HML: ",round(mod1.params[3],2))

portfolio # 1 beta_M:  1.29 beta_SMB:  1.45 beta_HML:  0.43
portfolio # 2 beta_M:  1.08 beta_SMB:  1.53 beta_HML:  0.23
portfolio # 3 beta_M:  1.05 beta_SMB:  1.25 beta_HML:  0.52
portfolio # 4 beta_M:  0.95 beta_SMB:  1.21 beta_HML:  0.59
portfolio # 5 beta_M:  0.99 beta_SMB:  1.31 beta_HML:  0.91
portfolio # 6 beta_M:  1.08 beta_SMB:  1.12 beta_HML:  -0.22
portfolio # 7 beta_M:  1.01 beta_SMB:  0.97 beta_HML:  0.15
portfolio # 8 beta_M:  0.98 beta_SMB:  0.84 beta_HML:  0.35
portfolio # 9 beta_M:  0.97 beta_SMB:  0.83 beta_HML:  0.56
portfolio # 10 beta_M:  1.07 beta_SMB:  0.9 beta_HML:  0.89
portfolio # 11 beta_M:  1.12 beta_SMB:  0.82 beta_HML:  -0.23
portfolio # 12 beta_M:  1.02 beta_SMB:  0.5 beta_HML:  0.05
portfolio # 13 beta_M:  0.99 beta_SMB:  0.45 beta_HML:  0.32
portfolio # 14 beta_M:  0.99 beta_SMB:  0.46 beta_HML:  0.56
portfolio # 15 beta_M:  1.12 beta_SMB:  0.59 beta_HML:  0.88
portfolio # 16 beta_M:  1.08 beta_SMB:  0.32 beta_HML:  -0.35
portfolio # 17 beta_M:  1.02 bet

In [ ]:
# part 2 - Equation 1. Modified from HW 3
# obtain ln(size)
avg_size = []
for column in s2_size:
    avg_size.append(s2_size[column].mean())
ln_avg_size = np.log(avg_size) # len = 25

# obtain ln(BEME)
avg_BEME = []
for column in s2_BEME:
    avg_BEME.append(s2_BEME[column].mean())
ln_avg_BEME = np.log(avg_BEME)

# OLS regression on avg_return and beta / ln(size) / ln(BEME)
s2_return_s1.iloc[:, 0:25] = s2_return_s1.iloc[:, 0:25].subtract(s2_return_s1['RF'],axis = 0)

avg_return = pd.DataFrame(s2_return_s1.iloc[:, 0:25].mean())
avg_return.columns = ['avg_return']

avg_return['ln(size)'] = ln_avg_size
avg_return['ln(BEME)'] = ln_avg_BEME
avg_return['beta_M'] = beta_M
x = sm.add_constant(avg_return[["ln(size)", "ln(BEME)","beta_M"]])

mod_eq1 = sm.OLS(avg_return["avg_return"], x).fit()
print("coefficient->","gamma_0: ",round(mod_eq1.params[0],2), "gamma_size: ", round(mod_eq1.params[1],2),\
      "gamma_B/M: ",round(mod_eq1.params[2],2), "gamma_M", round(mod_eq1.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq1.bse[0],2), "gamma_size: ", round(mod_eq1.bse[1],2),\
      "gamma_B/M: ",round(mod_eq1.bse[2],2), "gamma_M", round(mod_eq1.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq1.tvalues[0],2), "gamma_size: ", round(mod_eq1.tvalues[1],2),\
      "gamma_B/M: ",round(mod_eq1.tvalues[2],2), "gamma_M", round(mod_eq1.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq1.pvalues[0],2), "gamma_size: ", round(mod_eq1.pvalues[1],2),\
      "gamma_B/M: ",round(mod_eq1.pvalues[2],2), "gamma_M", round(mod_eq1.pvalues[3],2))

coefficient-> gamma_0:  1.82 gamma_size:  -0.04 gamma_B/M:  0.22 gamma_M -0.63
Standard Error-> gamma_0:  0.25 gamma_size:  0.01 gamma_B/M:  0.02 gamma_M 0.23
t-stat-> gamma_0:  7.22 gamma_size:  -4.27 gamma_B/M:  9.11 gamma_M -2.77
p-value-> gamma_0:  0.0 gamma_size:  0.0 gamma_B/M:  0.0 gamma_M 0.01


In [ ]:
# part 2 - Equation 2
avg_return['beta_SMB'] = beta_SMB
avg_return['beta_HML'] = beta_HML

x = sm.add_constant(avg_return[["beta_M", "beta_SMB","beta_HML"]])

mod_eq2 = sm.OLS(avg_return["avg_return"], x).fit()
# mod_eq2.summary()
print("coefficient->", "gamma_0: ",round(mod_eq2.params[0],2), "gamma_M: ", round(mod_eq2.params[1],2),\
      "gamma_SMB: ",round(mod_eq2.params[2],2), "gamma_HML", round(mod_eq2.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq2.bse[0],2), "gamma_M: ", round(mod_eq2.bse[1],2),\
      "gamma_SMB: ",round(mod_eq2.bse[2],2), "gamma_HML", round(mod_eq2.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq2.tvalues[0],2), "gamma_M: ", round(mod_eq2.tvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.tvalues[2],2), "gamma_HML", round(mod_eq2.tvalues[3],2))
print("p value->", "gamma_0: ",round(mod_eq2.pvalues[0],2), "gamma_M: ", round(mod_eq2.pvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.pvalues[2],2), "gamma_HML", round(mod_eq2.pvalues[3],2))

coefficient-> gamma_0:  1.82 gamma_M:  -1.1 gamma_SMB:  0.13 gamma_HML 0.4
Standard Error-> gamma_0:  0.33 gamma_M:  0.32 gamma_SMB:  0.05 gamma_HML 0.06
t-stat-> gamma_0:  5.51 gamma_M:  -3.45 gamma_SMB:  2.77 gamma_HML 6.42
p value-> gamma_0:  0.0 gamma_M:  0.0 gamma_SMB:  0.01 gamma_HML 0.0


In [ ]:
# part 2 - Equation 3
x = sm.add_constant(avg_return[["beta_M", "ln(size)","ln(BEME)", "beta_SMB","beta_HML"]])
mod_eq3 = sm.OLS(avg_return["avg_return"], x).fit()
mod_eq3.summary()
print("coefficient->","gamma_0: ", round(mod_eq3.params[0],2), "gamma_M: ", round(mod_eq3.params[1],2),\
      "gamma_size: ", round(mod_eq1.params[2],2), "gamma_B/M: ",round(mod_eq1.params[3],2),\
      "gamma_SMB: ",round(mod_eq3.params[4],2), "gamma_HML", round(mod_eq3.params[5],2))
print("Standard Error->","gamma_0: ", round(mod_eq3.bse[0],2), "gamma_M: ", round(mod_eq3.bse[1],2),\
      "gamma_size: ", round(mod_eq1.bse[2],2), "gamma_B/M: ",round(mod_eq1.bse[3],2),\
      "gamma_SMB: ",round(mod_eq3.bse[4],2), "gamma_HML", round(mod_eq3.bse[5],2))
print("t-stat->","gamma_0: ", round(mod_eq3.tvalues[0],2), "gamma_M: ", round(mod_eq3.tvalues[1],2),\
      "gamma_size: ", round(mod_eq1.tvalues[2],2), "gamma_B/M: ",round(mod_eq1.tvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.tvalues[4],2), "gamma_HML", round(mod_eq3.tvalues[5],2))
print("p value->","gamma_0: ", round(mod_eq3.pvalues[0],2), "gamma_M: ", round(mod_eq3.pvalues[1],2),\
      "gamma_size: ", round(mod_eq1.pvalues[2],2), "gamma_B/M: ",round(mod_eq1.pvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.pvalues[4],2), "gamma_HML", round(mod_eq3.pvalues[5],2))

coefficient-> gamma_0:  2.34 gamma_M:  -0.06 gamma_size:  0.22 gamma_B/M:  -0.63 gamma_SMB:  -0.39 gamma_HML -0.35
Standard Error-> gamma_0:  0.37 gamma_M:  0.27 gamma_size:  0.02 gamma_B/M:  0.23 gamma_SMB:  0.16 gamma_HML 0.14
t-stat-> gamma_0:  6.36 gamma_M:  -0.21 gamma_size:  9.11 gamma_B/M:  -2.77 gamma_SMB:  -2.39 gamma_HML -2.56
p value-> gamma_0:  0.0 gamma_M:  0.84 gamma_size:  0.0 gamma_B/M:  0.01 gamma_SMB:  0.03 gamma_HML 0.02


(Interesting sidenote from StackOverFlow- why SE from sm.ols results is called bse: The b is a historical artifact, when params where called b as in linear model y = Xb + u.) (that's interesting!)

## d)	Based on your estimates of these parameters in equations (1), (2), and (3), do characteristics or covariances better capture the cross-sectional variation in average returns?  What arguments can you make in favor of one side versus the other?  What arguments can you make that suggest you can’t tell which side is better able to capture returns?

There are arguments for both of using characterstics and covariances to capture the cross-sectional variation in average returns. Using covariance, we have taken into account the correlation among securities. However, characteristics give more intuitive relationship between return and risk; additionally, covariance is constant correlation, which characteristics does not assume.

In fact, based on parameters in equation (1), (2), and (3), characteristics give statistically significant estimatinos of the factors, with very high t-stat / low p-values, except for $\gamma_M$ for equation 3, indicating that the factor loads are almost all meaningful.

## e)	Repeat part c) using only data after January 1963.  Does your answer change in terms of whether characteristics or covariances better capture the cross-section of returns?  Do you now feel more or less strongly about your answer to this question in general?

In [ ]:
# Part e)
# Repeat Part c), but with data after January 1963

s2_return_s1_1963 = s2_return_s1[438:]
s2_size_1963 = s2_size[438:]
s2_BEME_1963 = s2_BEME[37:]

# part 1
beta_M = []
beta_SMB = []
beta_HML = []
for idx,i in enumerate(s2_return_s1_1963.columns[0:25]):
  y = s2_return_s1_1963[i] - s2_return_s1_1963["RF"]
  x = sm.add_constant(s2_return_s1_1963[["Mkt-RF", "SMB","HML"]])
  mod1 = sm.OLS(y,x,missing = 'drop').fit()
  beta_M.append(mod1.params[1])
  beta_SMB.append(mod1.params[2])
  beta_HML.append(mod1.params[3])
  print("portfolio #", idx+1, "beta_M: ",round(mod1.params[1],2), \
        "beta_SMB: ", round(mod1.params[2],2), "beta_HML: ",round(mod1.params[3],2))

portfolio # 1 beta_M:  1.08 beta_SMB:  1.37 beta_HML:  -0.31
portfolio # 2 beta_M:  0.97 beta_SMB:  1.3 beta_HML:  0.04
portfolio # 3 beta_M:  0.92 beta_SMB:  1.1 beta_HML:  0.28
portfolio # 4 beta_M:  0.89 beta_SMB:  1.04 beta_HML:  0.45
portfolio # 5 beta_M:  0.98 beta_SMB:  1.09 beta_HML:  0.69
portfolio # 6 beta_M:  1.11 beta_SMB:  0.99 beta_HML:  -0.4
portfolio # 7 beta_M:  1.01 beta_SMB:  0.87 beta_HML:  0.13
portfolio # 8 beta_M:  0.97 beta_SMB:  0.77 beta_HML:  0.38
portfolio # 9 beta_M:  0.97 beta_SMB:  0.73 beta_HML:  0.56
portfolio # 10 beta_M:  1.09 beta_SMB:  0.87 beta_HML:  0.81
portfolio # 11 beta_M:  1.09 beta_SMB:  0.73 beta_HML:  -0.45
portfolio # 12 beta_M:  1.05 beta_SMB:  0.54 beta_HML:  0.18
portfolio # 13 beta_M:  1.0 beta_SMB:  0.44 beta_HML:  0.44
portfolio # 14 beta_M:  1.0 beta_SMB:  0.4 beta_HML:  0.61
portfolio # 15 beta_M:  1.06 beta_SMB:  0.55 beta_HML:  0.77
portfolio # 16 beta_M:  1.06 beta_SMB:  0.38 beta_HML:  -0.43
portfolio # 17 beta_M:  1.08 beta_S

In [ ]:
# part 2 - Equation 1. Modified from HW 3
# obtain ln(size)
avg_size = []
for column in s2_size_1963:
    avg_size.append(s2_size_1963[column].mean())
ln_avg_size = np.log(avg_size) # len = 25

# obtain ln(BEME)
avg_BEME = []
for column in s2_BEME_1963:
    avg_BEME.append(s2_BEME_1963[column].mean())
ln_avg_BEME = np.log(avg_BEME)

# OLS regression on avg_return and beta / ln(size) / ln(BEME)
s2_return_s1_1963.iloc[:, 0:25] = s2_return_s1_1963.iloc[:, 0:25].subtract(s2_return_s1_1963['RF'],axis = 0)

avg_return = pd.DataFrame(s2_return_s1_1963.iloc[:, 0:25].mean())
avg_return.columns = ['avg_return']

avg_return['ln(size)'] = ln_avg_size
avg_return['ln(BEME)'] = ln_avg_BEME
avg_return['beta_M'] = beta_M
x = sm.add_constant(avg_return[["ln(size)", "ln(BEME)","beta_M"]])

mod_eq1 = sm.OLS(avg_return["avg_return"], x).fit()
print("coefficient->","gamma_0: ",round(mod_eq1.params[0],2), "gamma_size: ", round(mod_eq1.params[1],2),\
      "gamma_B/M: ",round(mod_eq1.params[2],2), "gamma_M", round(mod_eq1.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq1.bse[0],2), "gamma_size: ", round(mod_eq1.bse[1],2),\
      "gamma_B/M: ",round(mod_eq1.bse[2],2), "gamma_M", round(mod_eq1.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq1.tvalues[0],2), "gamma_size: ", round(mod_eq1.tvalues[1],2),\
      "gamma_B/M: ",round(mod_eq1.tvalues[2],2), "gamma_M", round(mod_eq1.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq1.pvalues[0],2), "gamma_size: ", round(mod_eq1.pvalues[1],2),\
      "gamma_B/M: ",round(mod_eq1.pvalues[2],2), "gamma_M", round(mod_eq1.pvalues[3],2))

coefficient-> gamma_0:  1.16 gamma_size:  -0.04 gamma_B/M:  0.24 gamma_M -0.42
Standard Error-> gamma_0:  0.39 gamma_size:  0.01 gamma_B/M:  0.04 gamma_M 0.39
t-stat-> gamma_0:  3.0 gamma_size:  -3.33 gamma_B/M:  6.54 gamma_M -1.09
p-value-> gamma_0:  0.01 gamma_size:  0.0 gamma_B/M:  0.0 gamma_M 0.29


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [ ]:
# part 2 - Equation 2
avg_return['beta_SMB'] = beta_SMB
avg_return['beta_HML'] = beta_HML

x = sm.add_constant(avg_return[["beta_M", "beta_SMB","beta_HML"]])

mod_eq2 = sm.OLS(avg_return["avg_return"], x).fit()
# mod_eq2.summary()
print("coefficient->", "gamma_0: ",round(mod_eq2.params[0],2), "gamma_M: ", round(mod_eq2.params[1],2),\
      "gamma_SMB: ",round(mod_eq2.params[2],2), "gamma_HML", round(mod_eq2.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq2.bse[0],2), "gamma_M: ", round(mod_eq2.bse[1],2),\
      "gamma_SMB: ",round(mod_eq2.bse[2],2), "gamma_HML", round(mod_eq2.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq2.tvalues[0],2), "gamma_M: ", round(mod_eq2.tvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.tvalues[2],2), "gamma_HML", round(mod_eq2.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq2.pvalues[0],2), "gamma_M: ", round(mod_eq2.pvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.pvalues[2],2), "gamma_HML", round(mod_eq2.pvalues[3],2))

coefficient-> gamma_0:  0.75 gamma_M:  -0.6 gamma_SMB:  0.17 gamma_HML 0.39
Standard Error-> gamma_0:  0.42 gamma_M:  0.41 gamma_SMB:  0.05 gamma_HML 0.06
t-stat-> gamma_0:  1.76 gamma_M:  -1.47 gamma_SMB:  3.35 gamma_HML 6.34
p-value-> gamma_0:  0.09 gamma_M:  0.16 gamma_SMB:  0.0 gamma_HML 0.0


In [ ]:
# part 2 - Equation 3
x = sm.add_constant(avg_return[["beta_M", "ln(size)","ln(BEME)", "beta_SMB","beta_HML"]])
mod_eq3 = sm.OLS(avg_return["avg_return"], x).fit()
mod_eq3.summary()
print("coefficient->","gamma_0: ", round(mod_eq3.params[0],2), "gamma_M: ", round(mod_eq3.params[1],2),\
      "gamma_size: ", round(mod_eq1.params[2],2), "gamma_B/M: ",round(mod_eq1.params[3],2),\
      "gamma_SMB: ",round(mod_eq3.params[4],2), "gamma_HML", round(mod_eq3.params[5],2))
print("Standard Error->","gamma_0: ", round(mod_eq3.bse[0],2), "gamma_M: ", round(mod_eq3.bse[1],2),\
      "gamma_size: ", round(mod_eq1.bse[2],2), "gamma_B/M: ",round(mod_eq1.bse[3],2),\
      "gamma_SMB: ",round(mod_eq3.bse[4],2), "gamma_HML", round(mod_eq3.bse[5],2))
print("t-stat->","gamma_0: ", round(mod_eq3.tvalues[0],2), "gamma_M: ", round(mod_eq3.tvalues[1],2),\
      "gamma_size: ", round(mod_eq1.tvalues[2],2), "gamma_B/M: ",round(mod_eq1.tvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.tvalues[4],2), "gamma_HML", round(mod_eq3.tvalues[5],2))
print("p-value->","gamma_0: ", round(mod_eq3.pvalues[0],2), "gamma_M: ", round(mod_eq3.pvalues[1],2),\
      "gamma_size: ", round(mod_eq1.pvalues[2],2), "gamma_B/M: ",round(mod_eq1.pvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.pvalues[4],2), "gamma_HML", round(mod_eq3.pvalues[5],2))

coefficient-> gamma_0:  1.77 gamma_M:  -0.16 gamma_size:  0.24 gamma_B/M:  -0.42 gamma_SMB:  -0.34 gamma_HML -0.16
Standard Error-> gamma_0:  0.68 gamma_M:  0.46 gamma_size:  0.04 gamma_B/M:  0.39 gamma_SMB:  0.31 gamma_HML 0.33
t-stat-> gamma_0:  2.61 gamma_M:  -0.34 gamma_size:  6.54 gamma_B/M:  -1.09 gamma_SMB:  -1.11 gamma_HML -0.49
p-value-> gamma_0:  0.02 gamma_M:  0.74 gamma_size:  0.0 gamma_B/M:  0.29 gamma_SMB:  0.28 gamma_HML 0.63


With data of 1963 and beyond, we now have more than a few low t-stat / high p-values, indicating the factors being statistically insignificant. Specifically:

$\gamma_M$ for equation 1

$\gamma_0$ and $\gamma_M$ for equation 2

$\gamma_M$, $\gamma_{B/M}$, $\gamma_{SMB}$, and $\gamma_{HML}$ for equation 3.

We feel less strongly about the characteristics comparing to covariance. This shows that in order for the characteristics to work, one needs to have a large enough amount of data. Even with monthly return data from 1963 to 2015, it is shown that the characteristics model would return less than statistically significant factors, and therefore put the reliability of it in question.

**25 Size and Momentum portfolios.**

## f)	Consider the following Fama-MacBeth cross-sectional regressions: 
$$
(1) \quad \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\mathrm{size}} \ln (\text { size })+\gamma_{\mathrm{ret} 212}(\mathrm{ret} 212)+\eta_{\mathrm{ii}}
$$
$$
\text { (2) } \quad \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\mathrm{size}} \beta_{\mathrm{iSMB}}+\gamma_{\mathrm{UMD}} \beta_{\mathrm{iUMD}}+\eta_{\mathrm{i} 2}
$$
$$
(3) \quad \mathrm{R}_{\mathrm{i}}=\gamma_{0}+\gamma_{\mathrm{M}} \beta_{\mathrm{iM}}+\gamma_{\text {size }} \ln (\text { size })+\gamma_{\mathrm{ret} 212}(\mathrm{ret} 212)+\gamma_{\mathrm{SMB}} \beta_{\mathrm{iSMB}}+\gamma_{\mathrm{UMD}} \beta_{\mathrm{iUMD}} + \eta_{\mathrm{i} 2}
$$

where $\gamma_{0}, \gamma_{\mathrm{M}}, \gamma_{\text {size }}, \gamma_{\text {ret212 }}, \gamma_{\mathrm{SMB}}, \text { and } \gamma_{\mathrm{UMD}}$  are regression parameters. $
\beta_{\mathrm{iM}}, \beta_{\mathrm{iSMB}}, \text { and } \beta_{iUMD}
$ are betas with respect to the Fama-French factors SMB and UMD and size and ret212 are the average size and past 2-to-12-month return characteristics of the portfolio.
Estimate equations (1), (2), and (3) using the full sample of data and the Fama- MacBeth procedure. The following is a brief outline of the procedure:
1. Estimate $\beta_{\mathrm{iM}}, \beta_{\mathrm{iSMB}}, \text { and } \beta_{iUMD}$ for each portfolio by running a time-series regression for each of the 25 portfolios on the Fama-French factors RMRF, SMB, and UMD. Assume that the betas do not change over time; hence, you can estimate the betas using full-period OLS regressions.
2. Each month estimate the regressions using the month-by-month cross-section of realized returns on the 25 portfolios on their estimated betas and characteristics according to equations (1), (2), and (3).
3. Compute the time series average of the estimates of $\gamma_{0}, \gamma_{\mathrm{M}}, \gamma_{\text {size }}, \gamma_{\text {ret212 }}, \gamma_{\mathrm{SMB}}, \text { and } \gamma_{\mathrm{UMD}}$  are regression parameters. $. In addition, compute the standard error and t-stat of the time series averages in the style of Fama and MacBeth (1973).


In [ ]:
# part 1
beta_M = []
beta_SMB = []
beta_UMD = []
for idx,i in enumerate(s3_return_s1.columns[0:25]):
  y = s3_return_s1[i] - s3_return_s1["RF"]
  x = sm.add_constant(s3_return_s1[["Mkt-RF", "SMB","UMD"]])
  mod1 = sm.OLS(y,x,missing = 'drop').fit()
  beta_M.append(mod1.params[1])
  beta_SMB.append(mod1.params[2])
  beta_UMD.append(mod1.params[3])
  print("portfolio #", idx+1, "beta_M: ",round(mod1.params[1],2), \
        "beta_SMB: ", round(mod1.params[2],2), "beta_UMD: ",round(mod1.params[3],2))

portfolio # 1 beta_M:  1.06 beta_SMB:  1.49 beta_UMD:  -0.76
portfolio # 2 beta_M:  1.01 beta_SMB:  1.31 beta_UMD:  -0.47
portfolio # 3 beta_M:  0.96 beta_SMB:  1.25 beta_UMD:  -0.31
portfolio # 4 beta_M:  1.04 beta_SMB:  1.27 beta_UMD:  -0.05
portfolio # 5 beta_M:  1.09 beta_SMB:  1.32 beta_UMD:  0.23
portfolio # 6 beta_M:  1.16 beta_SMB:  1.01 beta_UMD:  -0.71
portfolio # 7 beta_M:  1.02 beta_SMB:  0.91 beta_UMD:  -0.4
portfolio # 8 beta_M:  0.99 beta_SMB:  0.76 beta_UMD:  -0.18
portfolio # 9 beta_M:  1.01 beta_SMB:  0.89 beta_UMD:  0.04
portfolio # 10 beta_M:  1.15 beta_SMB:  1.01 beta_UMD:  0.33
portfolio # 11 beta_M:  1.18 beta_SMB:  0.61 beta_UMD:  -0.78
portfolio # 12 beta_M:  1.07 beta_SMB:  0.53 beta_UMD:  -0.4
portfolio # 13 beta_M:  1.01 beta_SMB:  0.5 beta_UMD:  -0.2
portfolio # 14 beta_M:  0.99 beta_SMB:  0.48 beta_UMD:  0.08
portfolio # 15 beta_M:  1.12 beta_SMB:  0.68 beta_UMD:  0.43
portfolio # 16 beta_M:  1.23 beta_SMB:  0.27 beta_UMD:  -0.83
portfolio # 17 beta_M:  1.

In [ ]:
# part 2 - Equation 1. Modified from HW 3
# obtain ln(size)
avg_size = []
for column in s3_size:
    avg_size.append(s3_size[column].mean())
ln_avg_size = np.log(avg_size) # len = 25

# obtain ret212
avg_ret212 = []
for column in s3_ret212:
    avg_ret212.append(s3_ret212[column].mean())

# OLS regression on avg_return and beta / ln(size) / ln(BEME)
s3_return_s1.iloc[:, 0:25] = s3_return_s1.iloc[:, 0:25].subtract(s3_return_s1['RF'],axis = 0)

avg_return = pd.DataFrame(s3_return_s1.iloc[:, 0:25].mean())
avg_return.columns = ['avg_return']

avg_return['ln(size)'] = ln_avg_size
avg_return['ret212'] = avg_ret212
avg_return['beta_M'] = beta_M
x = sm.add_constant(avg_return[["ln(size)", "ret212","beta_M"]])

mod_eq1 = sm.OLS(avg_return["avg_return"], x).fit()
print("coefficient->","gamma_0: ",round(mod_eq1.params[0],2), "gamma_size: ", round(mod_eq1.params[1],2),\
      "gamma_ret212: ",round(mod_eq1.params[2],2), "gamma_M", round(mod_eq1.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq1.bse[0],2), "gamma_size: ", round(mod_eq1.bse[1],2),\
      "gamma_ret212: ",round(mod_eq1.bse[2],2), "gamma_M", round(mod_eq1.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq1.tvalues[0],2), "gamma_size: ", round(mod_eq1.tvalues[1],2),\
      "gamma_ret212: ",round(mod_eq1.tvalues[2],2), "gamma_M", round(mod_eq1.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq1.pvalues[0],2), "gamma_size: ", round(mod_eq1.pvalues[1],2),\
      "gamma_ret212: ",round(mod_eq1.pvalues[2],2), "gamma_M", round(mod_eq1.pvalues[3],2))

coefficient-> gamma_0:  2.71 gamma_size:  -0.1 gamma_ret212:  0.01 gamma_M -1.31
Standard Error-> gamma_0:  0.32 gamma_size:  0.01 gamma_ret212:  0.0 gamma_M 0.32
t-stat-> gamma_0:  8.57 gamma_size:  -9.29 gamma_ret212:  17.21 gamma_M -4.15
p-value-> gamma_0:  0.0 gamma_size:  0.0 gamma_ret212:  0.0 gamma_M 0.0


In [ ]:
# part 2 - Equation 2
avg_return['beta_SMB'] = beta_SMB
avg_return['beta_UMD'] = beta_UMD

x = sm.add_constant(avg_return[["beta_M", "beta_SMB","beta_UMD"]])

mod_eq2 = sm.OLS(avg_return["avg_return"], x).fit()
# mod_eq2.summary()
print("coefficient->", "gamma_0: ",round(mod_eq2.params[0],2), "gamma_M: ", round(mod_eq2.params[1],2),\
      "gamma_SMB: ",round(mod_eq2.params[2],2), "gamma_UMD", round(mod_eq2.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq2.bse[0],2), "gamma_M: ", round(mod_eq2.bse[1],2),\
      "gamma_SMB: ",round(mod_eq2.bse[2],2), "gamma_UMD", round(mod_eq2.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq2.tvalues[0],2), "gamma_M: ", round(mod_eq2.tvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.tvalues[2],2), "gamma_UMD", round(mod_eq2.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq2.pvalues[0],2), "gamma_M: ", round(mod_eq2.pvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.pvalues[2],2), "gamma_UMD", round(mod_eq2.pvalues[3],2))

coefficient-> gamma_0:  1.72 gamma_M:  -0.92 gamma_SMB:  0.43 gamma_UMD 0.78
Standard Error-> gamma_0:  0.44 gamma_M:  0.43 gamma_SMB:  0.05 gamma_UMD 0.06
t-stat-> gamma_0:  3.89 gamma_M:  -2.17 gamma_SMB:  8.58 gamma_UMD 12.4
p-value-> gamma_0:  0.0 gamma_M:  0.04 gamma_SMB:  0.0 gamma_UMD 0.0


In [ ]:
x = sm.add_constant(avg_return[["beta_M", "ln(size)","ret212", "beta_SMB","beta_UMD"]])
mod_eq3 = sm.OLS(avg_return["avg_return"], x).fit()
mod_eq3.summary()
print("coefficient->","gamma_0: ", round(mod_eq3.params[0],2), "gamma_M: ", round(mod_eq3.params[1],2),\
      "gamma_size: ", round(mod_eq1.params[2],2), "gamma_ret212: ",round(mod_eq1.params[3],2),\
      "gamma_SMB: ",round(mod_eq3.params[4],2), "gamma_UMD", round(mod_eq3.params[5],2))
print("Standard Error->","gamma_0: ", round(mod_eq3.bse[0],2), "gamma_M: ", round(mod_eq3.bse[1],2),\
      "gamma_size: ", round(mod_eq1.bse[2],2), "gamma_ret212: ",round(mod_eq1.bse[3],2),\
      "gamma_SMB: ",round(mod_eq3.bse[4],2), "gamma_UMD", round(mod_eq3.bse[5],2))
print("t-stat->","gamma_0: ", round(mod_eq3.tvalues[0],2), "gamma_M: ", round(mod_eq3.tvalues[1],2),\
      "gamma_size: ", round(mod_eq1.tvalues[2],2), "gamma_ret212: ",round(mod_eq1.tvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.tvalues[4],2), "gamma_UMD", round(mod_eq3.tvalues[5],2))
print("p-value->","gamma_0: ", round(mod_eq3.pvalues[0],2), "gamma_M: ", round(mod_eq3.pvalues[1],2),\
      "gamma_size: ", round(mod_eq1.pvalues[2],2), "gamma_ret212: ",round(mod_eq1.pvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.pvalues[4],2), "gamma_UMD", round(mod_eq3.pvalues[5],2))

coefficient-> gamma_0:  3.38 gamma_M:  -1.04 gamma_size:  0.01 gamma_ret212:  -1.31 gamma_SMB:  -0.32 gamma_UMD 0.28
Standard Error-> gamma_0:  0.44 gamma_M:  0.3 gamma_size:  0.0 gamma_ret212:  0.32 gamma_SMB:  0.16 gamma_UMD 0.15
t-stat-> gamma_0:  7.72 gamma_M:  -3.52 gamma_size:  17.21 gamma_ret212:  -4.15 gamma_SMB:  -2.04 gamma_UMD 1.83
p-value-> gamma_0:  0.0 gamma_M:  0.0 gamma_size:  0.0 gamma_ret212:  0.0 gamma_SMB:  0.06 gamma_UMD 0.08


## g) Based on your estimates of these parameters in equations (1), (2), and (3), do characteristics or covariances better capture the cross-sectional variation in average returns? What arguments can you make in favor of one side versus the other? What arguments can you make that suggest you can’t tell which side is better able to capture returns?


Similar to earlier, there are arguments for both of using characterstics and covariances to capture the cross-sectional variation in average returns. Using covariance, we have taken into account the correlation among securities. However, characteristics give more intuitive relationship between return and risk; additionally, covariance is constant correlation, which characteristics does not assume.

For the momentum portfolios, based on parameters in equation (1), (2), and (3), characteristics give statistically very significant (more significant thatn the size portfolios) estimatinos of the factors, with very high t-stat, indicating that the factor loads are all meaningful, except, interestingly, the UMD loads in regression (3).

## h) Repeat part f) using only data after January, 1963. Does your answer change in terms of whether characteristics or covariances better capture the cross-section of returns? Do you now feel more or less strongly about your answer to this question in general?

In [ ]:
# part 1
s3_return_s1_1963 = s3_return_s1[438:]
s3_size_1963 = s2_size[438:]
s3_ret212_1963 = s3_ret212[438:]

beta_M = []
beta_SMB = []
beta_UMD = []
for idx,i in enumerate(s3_return_s1_1963.columns[0:25]):
  y = s3_return_s1_1963[i] - s3_return_s1_1963["RF"]
  x = sm.add_constant(s3_return_s1_1963[["Mkt-RF", "SMB","UMD"]])
  mod1 = sm.OLS(y,x,missing = 'drop').fit()
  beta_M.append(mod1.params[1])
  beta_SMB.append(mod1.params[2])
  beta_UMD.append(mod1.params[3])
  print("portfolio #", idx+1, "beta_M: ",round(mod1.params[1],2), \
        "beta_SMB: ", round(mod1.params[2],2), "beta_UMD: ",round(mod1.params[3],2))

portfolio # 1 beta_M:  1.02 beta_SMB:  1.22 beta_UMD:  -0.72
portfolio # 2 beta_M:  0.82 beta_SMB:  0.92 beta_UMD:  -0.32
portfolio # 3 beta_M:  0.8 beta_SMB:  0.83 beta_UMD:  -0.14
portfolio # 4 beta_M:  0.83 beta_SMB:  0.87 beta_UMD:  0.03
portfolio # 5 beta_M:  1.01 beta_SMB:  1.12 beta_UMD:  0.26
portfolio # 6 beta_M:  1.17 beta_SMB:  0.95 beta_UMD:  -0.74
portfolio # 7 beta_M:  0.93 beta_SMB:  0.73 beta_UMD:  -0.36
portfolio # 8 beta_M:  0.89 beta_SMB:  0.62 beta_UMD:  -0.11
portfolio # 9 beta_M:  0.91 beta_SMB:  0.71 beta_UMD:  0.03
portfolio # 10 beta_M:  1.13 beta_SMB:  0.94 beta_UMD:  0.35
portfolio # 11 beta_M:  1.15 beta_SMB:  0.61 beta_UMD:  -0.77
portfolio # 12 beta_M:  0.97 beta_SMB:  0.43 beta_UMD:  -0.36
portfolio # 13 beta_M:  0.91 beta_SMB:  0.42 beta_UMD:  -0.19
portfolio # 14 beta_M:  0.94 beta_SMB:  0.38 beta_UMD:  0.05
portfolio # 15 beta_M:  1.13 beta_SMB:  0.7 beta_UMD:  0.4
portfolio # 16 beta_M:  1.17 beta_SMB:  0.31 beta_UMD:  -0.81
portfolio # 17 beta_M:  1.

In [ ]:
# part 2 - Equation 1. Modified from HW 3
# obtain ln(size)
avg_size = []
for column in s3_size_1963:
    avg_size.append(s3_size_1963[column].mean())
ln_avg_size = np.log(avg_size) # len = 25

# obtain ret212
avg_ret212 = []
for column in s3_ret212_1963:
    avg_ret212.append(s3_ret212_1963[column].mean())

# OLS regression on avg_return and beta / ln(size) / ln(BEME)
s3_return_s1_1963.iloc[:, 0:25] = s3_return_s1_1963.iloc[:, 0:25].subtract(s3_return_s1_1963['RF'],axis = 0)

avg_return = pd.DataFrame(s3_return_s1_1963.iloc[:, 0:25].mean())
avg_return.columns = ['avg_return']

avg_return['ln(size)'] = ln_avg_size
avg_return['ret212'] = avg_ret212
avg_return['beta_M'] = beta_M
x = sm.add_constant(avg_return[["ln(size)", "ret212","beta_M"]])

mod_eq1 = sm.OLS(avg_return["avg_return"], x).fit()
print("coefficient->","gamma_0: ",round(mod_eq1.params[0],2), "gamma_size: ", round(mod_eq1.params[1],2),\
      "gamma_ret212: ",round(mod_eq1.params[2],2), "gamma_M", round(mod_eq1.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq1.bse[0],2), "gamma_size: ", round(mod_eq1.bse[1],2),\
      "gamma_ret212: ",round(mod_eq1.bse[2],2), "gamma_M", round(mod_eq1.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq1.tvalues[0],2), "gamma_size: ", round(mod_eq1.tvalues[1],2),\
      "gamma_ret212: ",round(mod_eq1.tvalues[2],2), "gamma_M", round(mod_eq1.tvalues[3],2))
print("p-value->", "gamma_0: ",round(mod_eq1.pvalues[0],2), "gamma_size: ", round(mod_eq1.pvalues[1],2),\
      "gamma_ret212: ",round(mod_eq1.pvalues[2],2), "gamma_M", round(mod_eq1.pvalues[3],2))

coefficient-> gamma_0:  1.26 gamma_size:  -0.04 gamma_ret212:  0.01 gamma_M -0.84
Standard Error-> gamma_0:  0.19 gamma_size:  0.01 gamma_ret212:  0.0 gamma_M 0.2
t-stat-> gamma_0:  6.61 gamma_size:  -3.17 gamma_ret212:  14.85 gamma_M -4.13
p-value-> gamma_0:  0.0 gamma_size:  0.0 gamma_ret212:  0.0 gamma_M 0.0


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [ ]:
# part 2 - Equation 2
avg_return['beta_SMB'] = beta_SMB
avg_return['beta_UMD'] = beta_UMD

x = sm.add_constant(avg_return[["beta_M", "beta_SMB","beta_UMD"]])

mod_eq2 = sm.OLS(avg_return["avg_return"], x).fit()
# mod_eq2.summary()
print("coefficient->", "gamma_0: ",round(mod_eq2.params[0],2), "gamma_M: ", round(mod_eq2.params[1],2),\
      "gamma_SMB: ",round(mod_eq2.params[2],2), "gamma_UMD", round(mod_eq2.params[3],2))
print("Standard Error->", "gamma_0: ",round(mod_eq2.bse[0],2), "gamma_M: ", round(mod_eq2.bse[1],2),\
      "gamma_SMB: ",round(mod_eq2.bse[2],2), "gamma_UMD", round(mod_eq2.bse[3],2))
print("t-stat->", "gamma_0: ",round(mod_eq2.tvalues[0],2), "gamma_M: ", round(mod_eq2.tvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.tvalues[2],2), "gamma_UMD", round(mod_eq2.tvalues[3],2))
print("p-stat->", "gamma_0: ",round(mod_eq2.pvalues[0],2), "gamma_M: ", round(mod_eq2.pvalues[1],2),\
      "gamma_SMB: ",round(mod_eq2.pvalues[2],2), "gamma_UMD", round(mod_eq2.pvalues[3],2))

coefficient-> gamma_0:  0.69 gamma_M:  -0.38 gamma_SMB:  0.24 gamma_UMD 0.78
Standard Error-> gamma_0:  0.28 gamma_M:  0.28 gamma_SMB:  0.07 gamma_UMD 0.08
t-stat-> gamma_0:  2.43 gamma_M:  -1.37 gamma_SMB:  3.38 gamma_UMD 9.73
p-stat-> gamma_0:  0.02 gamma_M:  0.19 gamma_SMB:  0.0 gamma_UMD 0.0


In [ ]:
# part 2 - Equation 3
x = sm.add_constant(avg_return[["beta_M", "ln(size)","ret212", "beta_SMB","beta_UMD"]])
mod_eq3 = sm.OLS(avg_return["avg_return"], x).fit()
mod_eq3.summary()
print("coefficient->","gamma_0: ", round(mod_eq3.params[0],2), "gamma_M: ", round(mod_eq3.params[1],2),\
      "gamma_size: ", round(mod_eq1.params[2],2), "gamma_ret212: ",round(mod_eq1.params[3],2),\
      "gamma_SMB: ",round(mod_eq3.params[4],2), "gamma_UMD", round(mod_eq3.params[5],2))
print("Standard Error->","gamma_0: ", round(mod_eq3.bse[0],2), "gamma_M: ", round(mod_eq3.bse[1],2),\
      "gamma_size: ", round(mod_eq1.bse[2],2), "gamma_ret212: ",round(mod_eq1.bse[3],2),\
      "gamma_SMB: ",round(mod_eq3.bse[4],2), "gamma_UMD", round(mod_eq3.bse[5],2))
print("t-stat->","gamma_0: ", round(mod_eq3.tvalues[0],2), "gamma_M: ", round(mod_eq3.tvalues[1],2),\
      "gamma_size: ", round(mod_eq1.tvalues[2],2), "gamma_ret212: ",round(mod_eq1.tvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.tvalues[4],2), "gamma_UMD", round(mod_eq3.tvalues[5],2))
print("p-value->","gamma_0: ", round(mod_eq3.pvalues[0],2), "gamma_M: ", round(mod_eq3.pvalues[1],2),\
      "gamma_size: ", round(mod_eq1.pvalues[2],2), "gamma_ret212: ",round(mod_eq1.pvalues[3],2),\
      "gamma_SMB: ",round(mod_eq3.pvalues[4],2), "gamma_UMD", round(mod_eq3.pvalues[5],2))

coefficient-> gamma_0:  1.65 gamma_M:  -0.7 gamma_size:  0.01 gamma_ret212:  -0.84 gamma_SMB:  -0.29 gamma_UMD -0.18
Standard Error-> gamma_0:  0.36 gamma_M:  0.44 gamma_size:  0.0 gamma_ret212:  0.2 gamma_SMB:  0.28 gamma_UMD 0.24
t-stat-> gamma_0:  4.56 gamma_M:  -1.58 gamma_size:  14.85 gamma_ret212:  -4.13 gamma_SMB:  -1.04 gamma_UMD -0.75
p-value-> gamma_0:  0.0 gamma_M:  0.13 gamma_size:  0.0 gamma_ret212:  0.0 gamma_SMB:  0.31 gamma_UMD 0.46


Again, with fewer data from just 1963 and beyond, we now have more than just one low t-stat. Specifically:

$\gamma_M$ for equation 2

$\gamma_M$, $\gamma_{SMB}$, and $\gamma_{UMD}$ for equation 3.

This again shows that we should feel less strongly about the characteristics comparing to covariance since one needs to have a large enough amount of data. Even with monthly return data from 1963 to 2015, it is shown that the characteristics model would return less than statistically significant factors, and therefore put the reliability of it in question.